In [9]:
import sqlite3
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.types import interrupt, Command

llm = init_chat_model("openai:gpt-4o-mini")

conn = sqlite3.connect(
    "memory.db",
    check_same_thread=False,
)

config = {
    "configurable": {
        "thread_id": "3",
    },
}

In [10]:
class State(MessagesState):
    custom_stuff: str


graph_builder = StateGraph(State)

In [11]:
@tool
def get_human_feedback(poem: str):
    """
    Asks the user for feedback on the poem.
    Use this before returning the final response.
    """
    feedback = interrupt(f"Here is the poem, tell me what you think\n{poem}")
    return feedback


llm_with_tools = llm.bind_tools(
    tools=[
        get_human_feedback,
    ]
)


def chatbot(state: State):
    response = llm_with_tools.invoke(
        f"""
        You are an expert in making poems.

        Use the `get_human_feedback` tool to get feedback on your poem.

        Only after you receive positive feedback you can return the final poem.

        ALWAYS ASK FOR FEEDBACK FIRST.

        Here is the conversation history:

        {state["messages"]}
    """
    )
    return {
        "messages": [response],
    }

In [12]:
tool_node = ToolNode(
    tools=[
        get_human_feedback,
    ],
)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)


graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")

graph = graph_builder.compile(
    checkpointer=SqliteSaver(conn),
)

In [13]:
result = graph.invoke(
    {
        "messages": [
            {"role": "user", "content": "Please make a poem about Python code."},
        ]
    },
    config=config,
)

In [14]:
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Please make a poem about Python code.
================================== Ai Message ==================================
Tool Calls:
  get_human_feedback (call_hQwx4mXFgpGKrZEalI2GVTdZ)
 Call ID: call_hQwx4mXFgpGKrZEalI2GVTdZ
  Args:
    poem: In a world of text and lines,
Python breathes, the code unwinds.
Beneath the syntax, logic grows,
In functions, loops, where knowledge flows.

Indentation, a sacred space,
Each line finds its rightful place.
With variables, we cast our dreams,
In lists and tuples, logic gleams.

From simple prints to complex tasks,
In every bracket, wonder basks.
Exceptions caught, errors tamed,
In Python's realms, the coder's framed.

Libraries vast, in modules we trust,
NumPy, Pandas, in coding we lust.
With every import, our toolkit expands,
Crafting solutions with deft, skilled hands.

So let’s embrace this coding art,
For Python’s here to play its part.
In every script, a chance 

In [15]:
snapshot = graph.get_state(config)

snapshot.next

('tools',)

In [16]:
response = Command(resume="It looks great!")

result = graph.invoke(
    response,
    config=config,
)
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Please make a poem about Python code.
================================== Ai Message ==================================
Tool Calls:
  get_human_feedback (call_hQwx4mXFgpGKrZEalI2GVTdZ)
 Call ID: call_hQwx4mXFgpGKrZEalI2GVTdZ
  Args:
    poem: In a world of text and lines,
Python breathes, the code unwinds.
Beneath the syntax, logic grows,
In functions, loops, where knowledge flows.

Indentation, a sacred space,
Each line finds its rightful place.
With variables, we cast our dreams,
In lists and tuples, logic gleams.

From simple prints to complex tasks,
In every bracket, wonder basks.
Exceptions caught, errors tamed,
In Python's realms, the coder's framed.

Libraries vast, in modules we trust,
NumPy, Pandas, in coding we lust.
With every import, our toolkit expands,
Crafting solutions with deft, skilled hands.

So let’s embrace this coding art,
For Python’s here to play its part.
In every script, a chance 

In [17]:
snapshot = graph.get_state(config)

snapshot.next

()